In [1]:
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry
import time

In [3]:
df = pd.read_csv('data/aod_sample_500.csv')
muestra = df.sample(n=5, random_state=42)
muestra.reset_index(drop=True, inplace=True)
muestra.head()

,Date,Latitude,Longitude
0,2007-05-10,-25.898930,139.345963
1,2018-11-28,-23.273001,16.502945
2,2016-05-07,67.366623,26.629570
3,2016-10-15,38.567833,-7.911500
4,2023-05-26,55.906799,-4.531220


In [ ]:
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

variables = [
    "temperature_2m_mean",
    "sunshine_duration",
    "wind_speed_10m_max",
    "wind_direction_10m_dominant",
    "shortwave_radiation_sum",
    "et0_fao_evapotranspiration",
    "relative_humidity_2m_mean"
]

resultados = []
count = 1
for idx, fila in muestra.iterrows():
    lat = fila["Latitude"]
    lon = fila["Longitude"]
    fecha = pd.to_datetime(fila["Date"]).strftime("%Y-%m-%d")  # formato correcto para la API

    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": fecha,
        "end_date": fecha,
        "daily": variables
    }

    try:
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]

        daily = response.Daily()
        data = {
            "Date": fecha,
            "Latitude": lat,
            "Longitude": lon,
            "temperature_2m_mean": daily.Variables(0).ValuesAsNumpy()[0],
            "sunshine_duration": daily.Variables(1).ValuesAsNumpy()[0],
            "wind_speed_10m_max": daily.Variables(2).ValuesAsNumpy()[0],
            "wind_direction_10m_dominant": daily.Variables(3).ValuesAsNumpy()[0],
            "shortwave_radiation_sum": daily.Variables(4).ValuesAsNumpy()[0],
            "et0_fao_evapotranspiration": daily.Variables(5).ValuesAsNumpy()[0],
            "relative_humidity_2m_mean": daily.Variables(6).ValuesAsNumpy()[0]
        }

        resultados.append(data)
        print(f"Petición {count} completado")
        count += 1
        time.sleep(2)

    except Exception as e:
        print(f"Error al procesar fila {idx} ({lat}, {lon}, {fecha}): {e}")

df_final = pd.DataFrame(resultados)

print("DataFrame final con resultados meteorológicos:")
df_final

Petición 0 completado
Petición 0 completado
Petición 0 completado
Petición 0 completado
Petición 0 completado
DataFrame final con resultados meteorológicos:


,Date,Latitude,Longitude,temperature_2m_mean,sunshine_duration,wind_speed_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,relative_humidity_2m_mean
0,2007-05-10,-25.898930,139.345963,19.321333,36180.843750,23.773632,152.841003,14.790000,3.747578,64.834335
1,2018-11-28,-23.273001,16.502945,21.236916,45434.101562,25.264202,102.063797,32.049999,8.448574,27.577719
2,2016-05-07,67.366623,26.629570,5.406000,55794.078125,12.434340,240.960388,22.420000,2.616398,70.835083
3,2016-10-15,38.567833,-7.911500,15.593583,37164.613281,8.707238,217.214340,15.590000,2.685369,73.763474
4,2023-05-26,55.906799,-4.531220,12.323750,42194.976562,13.450708,298.619965,17.500000,2.688250,80.609184
